## Set-up

In [321]:
# import libraries
from IPython.display import JSON #REMOVE ONCE NOTEBOOK IS COMPLETE
import requests
import pandas as pd
import os
import time

In [ ]:
# set constants:
API_KEY = os.getenv('FOURSQUARE_API_KEY')
CALL_DELAY = 0.5 # pause (in s) after each call

# Foursquare

Notes:
- !! There does not need to be a seperate call for each fsq_id, the getPOIDetails_FSQR() function needs to be assimilated into getPOIs_FSQR() !!
- Need to adjust city_bikes.ipynb to provide name and address of bike station (more valuable than lat,long as an output)
    - Then add processing for name and address of bike stations
- Should add call_delay

In [353]:
# load data
palmaBikes = pd.read_csv('Statistical-Modelling/data/palmaBikes.csv')

In [405]:
# define functions

# general
def getLatLong(cityBikes_df):
    ll_list = []
    for station in cityBikes_df.index:
        ll_list.append('{},{}'.format(str(cityBikes_df.iloc[station]['latitude']), str(cityBikes_df.iloc[station]['longitude'])))
    return ll_list

# Foursquare
def getPOIDetails_FSQR(fsq_id):
    # make API call (seleted fields are 'rating', 'price', 'stats', 'popularity', 'features')
    url = "https://api.foursquare.com/v3/places/{}?fields=rating%2Cprice%2Cstats%2Cpopularity%2Cfeatures".format(fsq_id)

    headers = {
        "accept": "application/json",
        "Authorization": API_KEY
    }
    
    response = requests.get(url, headers=headers).json()
    
    # collect rating, popularity, ratings, total_stats (if they exist)
    try:
        rating = response['rating']
    except:
        rating = None
    try:
        popularity = response['popularity']
    except:
        popularity = None
    try:
        ratings = response['stats']['total_ratings']
    except:
        ratings = None
    try:
        total_stats = response['stats']['total_photos'] + response['stats']['total_ratings'] + response['stats']['total_tips']
    except:
        total_stats = 'NA'
    
    return rating, popularity, ratings, total_stats
    
def getPOIs_FSQR(cityBikes_df):
    
    # format latitude and longitude for Foursquare API
    cityBikes_df['ll'] = (pd.Series(getLatLong(palmaBikes)))
    
    for station in cityBikes_df.index:
        # make API call
        url = "https://api.foursquare.com/v3/places/search"

        params = {
            "ll": cityBikes_df.iloc[station]['ll'],
            "radius": "1000", #1km
            "limit": "10", #Brought down from 50 to 10 for testing
            "sort":"DISTANCE" #Might be useful, but I'm not sure how yet
        }

        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY
        }

        response = requests.request("GET", url, params=params, headers=headers).json()
        
        df = pd.DataFrame(columns = ['category', 'distance', 'fsq_id', 'name', 'rating', 'popularity', 'ratings', 'total_stats'])
        
        for POI in response['results']:
            # collecting category, distance, fsq_id, name
            category = POI['categories'][0]['name']
            distance = POI['distance']
            fsq_id = POI['fsq_id']
            name = POI['name']

            #collecting rating, popularity, ratings, total_stats
            rating, popularity, ratings, total_stats = getPOIDetails_FSQR(fsq_id)

            # save to dataframe
            df = df.append({'category': category, 'distance': distance, 
                            'fsq_id': fsq_id, 'name': name, 'rating': rating,
                            'popularity': popularity, 'ratings':ratings,
                            'total_stats': total_stats}, ignore_index=True)
        
        return df
        


# Yelp

In [406]:
getPOIs_FSQR(palmaBikes)

/var/folders/k6/sjx5jh0s4ps0mm1lbstv_yrm0000gn/T/ipykernel_41679/957875772.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'category': category, 'distance': distance,
/var/folders/k6/sjx5jh0s4ps0mm1lbstv_yrm0000gn/T/ipykernel_41679/957875772.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'category': category, 'distance': distance,
/var/folders/k6/sjx5jh0s4ps0mm1lbstv_yrm0000gn/T/ipykernel_41679/957875772.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'category': category, 'distance': distance,
/var/folders/k6/sjx5jh0s4ps0mm1lbstv_yrm0000gn/T/ipykernel_41679/957875772.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

,category,distance,fsq_id,name,rating,popularity,ratings,total_stats
0,Farmers' Market,43,4bc84d218b7c9c748cd337cf,Mercat de Pere Garau,8.4,0.998674,75,115
1,Café,108,4d331da3329e54811bbcb61d,Churreria la Artesana,6.2,0.886616,8,23
2,Pedestrian Plaza,205,5379daef11d27506c0598b8c,Plaça de Pere Garau,None,0.979232,None,NA
3,Home Service,298,579f1a55498e79d91685057a,Bongrup Baleares,None,0.250365,None,NA
4,Taco Restaurant,299,4f5e6a0ce4b0028cf960560b,Chapultepec,9.0,0.971190,77,156
5,Health and Beauty Service,315,5d96fa77f6b75b000716362f,D2BD Lounge,None,0.258672,None,NA
6,Grocery Store / Supermarket,326,4cfe90b759c7b60c34f29f77,Mercadona,5.6,0.975211,9,14
7,Asian Restaurant,331,4d3de317af8ca1436f0110f7,Shi Shan Xuan,8.0,0.952410,120,231
8,Bookstore,334,50b4af8de4b047c32be47ff1,Papeleria Clips,None,0.210950,None,NA
9,Bank,358,4fb3c9ef7beba00e2f48c301,BBVA,6.1,0.895586,0,NA


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating